In [16]:
# Setting dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import numpy as np
import time
import re
import warnings
from splinter import Browser

In [17]:
# Setting path to chromedriver
executable_path = {'executable_path': '/Users/konstajokipii/NU_BOOTCAMP/web-scraping-challenge/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
# Visiting Mars news URL
NasaMarsNews = 'https://mars.nasa.gov/news/'
browser.visit(NasaMarsNews)

# Creating HTML object
html = browser.html

# Parsing HTML w/ BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Getting the first item in a list <li> under an unordered list <ul> 
latest_news = soup.find('li', class_='slide')

# Saving the news item under a <div> container
news_title = latest_news.find('div', class_='content_title').text
print(news_title)

# Saving the text paragraph in the container with an 'article_teaser_body' class
news_paragraph = latest_news.find('div', class_='article_teaser_body').text
print(news_paragraph)

NASA's Mars Helicopter to Make First Flight Attempt Sunday
The small rotorcraft’s “Wright brothers moment” is two Mars days away.


In [19]:
# Visiting Mars facts URL
MarsFacts = 'https://space-facts.com/mars/'
browser.visit(MarsFacts)

# Creating HTML object
html = browser.html

# Using pandas to scrape the table
table = pd.read_html(html)

# Slicing the table into a dataframe
marsfacts_df = table[0]
marsfacts_df.columns =['Description', 'Value']
marsfacts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [20]:
# Visiting USGS Astrogeology URL
MarsHemisphere = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(MarsHemisphere)

# Creating HTML object
html = browser.html

# Parsing HTML w/ BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Retrieing parent containers for each hemisphere 
hemispheres = soup.find_all('div', class_="item")

# Creating empty list for storing the python dictionary
hemisphere_image_data = []

# For loop through each container
for hemisphere in range(len(hemispheres)):

    # Using Splinter to click on all links to get image data
    hemisphere_url = browser.find_by_css("a.product-item h3")
    hemisphere_url[hemisphere].click()
    
    # Creating a BeautifulSoup object with the image URL
    image_url = browser.html
    image_soup = BeautifulSoup(image_url, 'html.parser')
    
    # Storing prefix URL for fullsize image links
    prefix_url = 'https://astrogeology.usgs.gov'
    
    # Saving full resolution images into variable
    suffix_url = image_soup.find('img', class_="wide-image")['src']
    
    # Joining URLs
    full_image_url = prefix_url + suffix_url

    # Saving image title into a variable
    image_title = browser.find_by_css('.title').text
    
    # Adding key value pairs to python dictionary and appending to list
    hemisphere_image_data.append({"title": image_title,
                              "img_url": full_image_url})
    
    # Returning to main page
    browser.back()
    
# Closing browser session  
browser.quit()

hemisphere_image_data

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]